This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](catmull-rom.ipynb)

# Uniform Catmull-Rom Splines

In <cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>,
a class of splines is presented which can be,
in its most generic form,
described mathematically with what is referred to as equation (1):

\begin{equation*}
F(s) =
\frac{
\sum x_i(s) w_i(s)
}{
\sum w_i(s)
},
\end{equation*}
where the part $w_i(s)/\sum w_i(s)$ is called *blending functions*.

> Since the blending functions presented above are,
as of now, completely
arbitrary
we impose
some
constraints in order to make them easier to use. We
shall deal only with blending functions that are zero
outside of some given interval.
Also we require that
$\sum w_i(s)$ does not vanish for any $s$.
We shall normalize $w_i(s)$ so that
$\sum w_i(s) = 1$ for all $s$.
>
> ---<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>, section 3, "Blending Functions"

The components of the equation are further constrained
to produce a interpolating function:

> Consider the following case:
Let $x_i(s)$ be any function
interpolating the points $p_i$ through $p_{i+k}$
and let $w_i(s)$ be zero outside $(s_{i-1}, s_{i+k+1})$.
The function $F(s)$ defined in equation (1)
will thus be an interpolating function.
Intuitively, this says that if all of the functions
that have an effect at a point,
pass through the point,
then the average of the functions will pass through the point.
>
> ---<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>, section 2: "The Model"

<div class="alert alert-warning">

Typo Alert

The typo "$p_i$ through $s_{i+k}$" has been fixed in the quote above.

</div>

> A polynomial of degree $k$
that pass[e]s through $k+1$ points will be used as $x(s)$.
In general it will not pass through the other points.
If the width of the interval in which $w_i(s)$ is non zero
is less than or equal to $k+2$ then $x_i(s)$ will not affect $F(s)$
outside the interpolation interval.
This means that $F(s)$ will be an interpolating function.
On the other hand
if the width of $w_i(s)$ is greater than $k+2$
then $x_i(s)$ will have an effect on the curve
outside the interpolation interval.
$F(s)$ will then be an approximating function.
>
> ---<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>, section 2: "The Model"

After limiting the scope of the paper to *interpolating* splines,
it is further reduced to *uniform* splines:

> [...] in
the
parametric
space
we can, without
loss
of
generality, place $s_j=j$.
>
> ---<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>, section 2: "The Model"

Whether or not generality is lost,
this means that the rest of the paper doesn't give any hints
how to construct non-uniform splines.
For those who are interested anyway,
we show how to do that in
[the notebook about non-uniform Catmull--Rom splines](catmull-rom-non-uniform.ipynb)
and once again in
[the notebook about the Barry--Goldman algorithm](catmull-rom-barry-goldman.ipynb).

After the aforementioned constraints
and the definition of the term *cardinal function* ...

> Cardinal function: a function that is $1$ at some
knot, $0$ at all other knots and can be anything in
between the other knots. It satisfies $F_i(s_j) = \delta_{ij}$.
>
> ---<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>, section 1: "Introduction"

... the gratuitously generic equation (1) is made a bit more concrete:

> If in equation (1) we assume $x_i(s)$
to be polynomials of degree $k$
then this equation can be reduced to a much simpler form:
>
> \begin{equation*}
F(s) =
\sum_j p_j C_{jk}(s)
\end{equation*}
> where the $C_{jk}(s)$ are cardinal blending functions
and $j$ is the knot to which the cardinal function and the point belong
and each $C_{jk}(s)$ is a shifted version of $C_{0,k}(s)$.
$C_{0,k}(s)$ is a function of both the degree $k$
of the polynomials and the blending functions $w(s)$:
>
> \begin{equation*}
C_{0,k}(s) =
\sum_{i=0}^k
\Big[
\prod_{j=i-k\\j \ne 0}^i \left(\frac{s}{j}+1\right)
\Big]
w(s+i)
\end{equation*}
>
> In essence we see that for a polynomial case
our cardinal functions are a blend of Lagrange polynomials.
When calculating $C_{0,k}(s)$,
$w(s)$ should be centered about $\frac{k}{2}$.
>
> ---<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>, section 4: "Calculating Cardinal Functions"

This looks like something we can work with,
even though the blending function $w(s)$ is still not defined.

In [ ]:
import sympy as sp

We use $t$ instead of $s$:

In [ ]:
t = sp.symbols('t')

In [ ]:
i, j, k = sp.symbols('i j k')

In [ ]:
w = sp.Function('w')

In [ ]:
C0k = sp.Sum(
        sp.Product(
            sp.Piecewise((1, sp.Eq(j, 0)), ((t / j) + 1, True)),
            (j, i - k, i)) * w(t + i),
        (i, 0, k))
C0k

## Blending Functions

<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>
leaves the choice of blending function to the reader.
It shows two plots (figure 1 and figure 3) for a custom blending function
stitched together from two Bézier curves,
but it doesn't show the cardinal function nor an actual spline
created from it.

The only other concrete suggestion is to use B-spline basis functions
as blending functions.
A quadratic B-spline basis function is shown in figure 2
and both cardinal functions and example curves are shown
that utilize both quadratic and cubic B-spline basis functions
(figures 4 through 7).
No mathematical description of B-spline basis functions is given,
instead the paper refers to
<cite data-cite="gordon1974bspline">Gordon and Riesenfeld (1974)</cite>.
That paper provides a pair of equations (3.1 and 3.2)
that can be used to recursively construct B-spline basis functions.
Simplified to the *uniform* case,
this leads to the base case (degree 0) ...

In [ ]:
B0 = sp.Piecewise((0, t < i), (1, t < i + 1), (0, True))
B0

... which can be used to obtain the linear (degree 1) basis functions:

In [ ]:
B1 = (t - i) * B0 + (i + 2 - t) * B0.subs(i, i + 1)

We can use one of them (where $i = 0$) as blending function:

In [ ]:
w1 = B1.subs(i, 0)

With some helper functions from [helper.py](helper.py)
we can plot this.

In [ ]:
from helper import plot_sympy, grid_lines

In [ ]:
plot_sympy(w1, (t, -0.2, 2.2))
grid_lines([0, 1, 2], [0, 1])

The quadratic (degree 2) basis functions can be obtained like this:

In [ ]:
B2 = (t - i) / 2 * B1 + (i + 3 - t) / 2 * B1.subs(i, i + 1)

For our further calculations,
we use the function with $i=-1$ as blending function:

In [ ]:
w2 = B2.subs(i, -1)

In [ ]:
plot_sympy(w2, (t, -1.2, 2.2))
grid_lines([-1, 0, 1, 2], [0, 1])

This should be the same function as shown in figure 2 of
<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>.

## Cardinal Functions

The first example curve in the paper (figure 5)
is a cubic curve,
constructed using
a cardinal function with $k=1$ (i.e. using linear Lagrange interpolation)
and
a quadratic B-spline basis function (as shown above)
as blending function.

With the information so far,
we can construct the cardinal function $C_{0,1}(t)$,
using our *quadratic* B-spline blending function `w2`
(which is, as required, centered about $\frac{k}{2}$):

In [ ]:
C01 = C0k.subs(k, 1).replace(w, lambda x: w2.subs(t, x)).doit().simplify()
C01

In [ ]:
plot_sympy(C01, (t, -2.2, 2.2))
grid_lines(range(-2, 3), [0, 1])

This should be the same function as shown in figure 4 of
<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>.

The paper does not show that,
but we can also try to flip the respective degrees
of Lagrange interpolation and B-spline blending.
In other words, we can set $k=2$ to construct the
cardinal function $C_{0,2}(t)$,
this time using the *linear* B-spline blending function `w1`
(which is also centered about $\frac{k}{2}$)
leading to a total degree of 3:

In [ ]:
C02 = C0k.subs(k, 2).replace(w, lambda x: w1.subs(t, x)).doit().simplify()

And as it turns out, this is exactly the same thing!

In [ ]:
assert C01 == C02

By the way, we come to the same conclusion in our
[notebook about the Barry--Goldman algorithm](catmull-rom-barry-goldman.ipynb),
which means that this is also true in the *non-uniform* case.

Many authors nowadays, when using the term *Catmull--Rom spline*,
mean the cubic spline created using exactly this cardinal function.

As we have seen,
this can be equivalently understood
either as
three linear interpolations
(more exactly: one interpolation and two extrapolations)
followed by quadratic B-spline blending
or as
two overlapping quadratic Lagrange interpolations
followed by linear blending.

## Example Plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

To quickly check how a spline segment would look like
when using the cardinal function we just derived,
let's define a few points ...

In [ ]:
vertices = np.array([
    (-0.1, -0.5),
    (0, 0),
    (1, 0),
    (0.5, 1),
])

... and plot $F(t)$ (or $F(s)$, as it has been called originally):

In [ ]:
plt.scatter(*np.array([
    sum([vertices[i] * C01.subs(t, s - i + 1) for i in range(4)])
    for s in np.linspace(0, 1, 20)]).T)
plt.plot(*vertices.T, 'x:g');

For calculating more than one segment,
and also for creating non-uniform Catmull--Rom splines,
the class
[splines.CatmullRom](../python-module/splines.rst#splines.CatmullRom)
can be used.
For more plots, see
[the notebook about properties of Catmull--Rom splines](catmull-rom-properties.ipynb).

## Basis Polynomials

The piecewise expression for the cardinal function
is a bit unwieldy to work with,
so let's bring it into a form we know how to deal with.

We are splitting the piecewise expression into four separate pieces,
each one to be evaluated at $0 \le t \le 1$.
We are also reversing the order of the pieces,
to match our intended control point order:

In [ ]:
b_CR = sp.Matrix([
    expr.subs(t, t + cond.args[1] - 1)
    for expr, cond in C01.args[1:-1][::-1]]).T
b_CR.T

In [ ]:
from helper import plot_basis

In [ ]:
plot_basis(*b_CR, labels=sp.symbols('xbm_i-1 xbm_i xbm_i+1 xbm_i+2'))

For the following sections,
we are using a few tools from [utility.py](utility.py):

In [ ]:
from utility import NamedExpression, NamedMatrix

## Basis Matrix

In [ ]:
b_monomial = sp.Matrix([t**3, t**2, t, 1]).T
M_CR = NamedMatrix(r'{M_\text{CR}}', 4, 4)
control_points = sp.Matrix(sp.symbols('xbm3:7'))

As usual, we look at the fifth polynomial segment
(from $\boldsymbol{x}_4$ to $\boldsymbol{x}_5$):

In [ ]:
p4 = NamedExpression('pbm4', b_monomial * M_CR.name * control_points)
p4

From the basis polynomials and the control points,
we can already calculate $\boldsymbol{p}_4(t)$ ...

In [ ]:
p4.expr = b_CR.dot(control_points).expand().collect(t)
p4

... and with a little bit of squinting,
we can directly read off
the coefficients of the basis matrix:

In [ ]:
M_CR.expr = sp.Matrix([
    [b.get(m, 0) for b in [
        p4.expr.expand().coeff(cv).collect(t, evaluate=False)
        for cv in control_points]]
    for m in b_monomial])
M_CR.pull_out(sp.S.Half)

This matrix also appears in section 6 of
<cite data-cite="catmull1974splines">Catmull and Rom (1974)</cite>.

In case you want to copy&paste it, here's a plain text version:

In [ ]:
print(_.expr)

And, in case somebody needs it, its inverse looks like this:

In [ ]:
M_CR.I

In [ ]:
print(_.expr)

## Tangent Vectors

To get the tangent vectors,
we simply have to take the first derivative ...

In [ ]:
pd4 = p4.diff(t)

... and evaluate it at the beginning and the end of the segment:

In [ ]:
pd4.evaluated_at(t, 0)

In [ ]:
pd4.evaluated_at(t, 1)

These two expressions can be generalized to
(as already shown in
[the notebook about Catmull--Rom properties](catmull-rom-properties.ipynb#Tangent-Vectors)):

\begin{equation*}
\boldsymbol{\dot{x}}_i =
\frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2}
\end{equation*}